In [ ]:
import pandas as pd
import numpy as py
import matplotlib.pyplot as plt


In [ ]:
# Teammate Chris code begins here

In [ ]:
# Teammate Chris code ends here

In [ ]:
# Teammate Matt code begins here

In [ ]:
# Teammate Matt code ends here

In [ ]:
# Teammate Carrie code begins here

In [ ]:
# Teammate Carrie code ends here

In [ ]:
# Teammate Glenda code begins here

In [ ]:
# Teammate Glenda code ends here